In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import random
import math
import time
from torchtext import data
from transformers import T5Tokenizer, T5Model

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [4]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [5]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [6]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [7]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [8]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [9]:
SRC = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TRG = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

In [10]:
fields = [('src', SRC), ('trg', TRG)]

In [11]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'marco.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(4321))

In [12]:
print(len(train_data.examples))
print(len(valid_data.examples))

42464
867


In [13]:
print(vars(train_data.examples[10000]))

{'src': [2625, 3, 10, 8, 7982, 3247, 130, 2325, 13243, 19334, 5, 227, 335, 3539, 6, 3, 4606, 40, 232, 16903, 3, 9, 3805, 13243, 740, 2486, 30, 8, 5022, 13, 56, 23, 265, 8, 24025, 127, 5, 166, 6, 3, 51, 14708, 11, 3, 9441, 1306, 13243, 7, 130, 1192, 5, 728, 56, 23, 265, 141, 3, 16804, 2127, 112, 3356, 16, 3, 4606, 40, 232, 6, 3, 88, 1192, 1450, 3372, 453, 13243, 7, 5, 11417, 3, 10, 125, 13243, 7, 410, 8, 7982, 3247, 918, 11, 213], 'trg': [3, 51, 14708, 11, 3, 9441, 1306, 13243, 7, 6, 3, 4606, 40, 232, 5]}


In [14]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6000])['src'])

print(tokens)
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6000])['trg'])

print(tokens)

['▁context', '▁', ':', '▁from', '▁', 'wikipedia', ',', '▁the', '▁free', '▁', 'ency', 'clo', 'pedia', '.', '▁wind', 'ley', '▁key', '▁fossil', '▁reef', '▁ge', 'ological', '▁state', '▁park', '▁is', '▁', 'a', '▁flori', 'd', 'a', '▁state', '▁park', '▁located', '▁at', '▁mile', '▁marker', '▁8', '5.5', '▁near', '▁', 'islam', 'o', 'rada', '.', '▁it', '▁was', '▁', 'a', '▁former', '▁quarry', '▁used', '▁by', '▁', 'hen', 'ry', '▁flag', 'ler', '▁in', '▁the', '▁early', '▁1900', 's', '▁to', '▁help', '▁his', '▁building', '▁of', '▁the', '▁overseas', '▁railroad', '.', '▁following', '▁the', '▁railroad', "'", 's', '▁completion', ',', '▁it', '▁was', '▁', 'a', '▁source', '▁for', '▁decorative', '▁stone', '▁pieces', '▁called', '▁key', 'stone', '.', '▁query', '▁', ':', '▁where', '▁is', '▁wind', 'ley', '▁key']
['▁wind', 'ley', '▁key', '▁fossil', '▁reef', '▁ge', 'ological', '▁state', '▁park', '▁is', '▁', 'a', '▁flori', 'd', 'a', '▁state', '▁park', '▁located', '▁at', '▁mile', '▁marker', '▁8', '5.5', '▁near', '▁', 

In [15]:
device = torch.device('cuda')

BATCH_SIZE = 20

train_iterator, valid_iterator = BucketIterator.splits(
                                (train_data, valid_data), 
                                batch_size = BATCH_SIZE,
                                device = device,
                                sort_key=lambda x: len(x.src))

In [16]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(t5.config.to_dict()['d_model'], t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src, decoder_input_ids=trg)
        
        output = self.out(embedded[0])
        
        return output

In [17]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [19]:
LEARNING_RATE = 0.0004

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [20]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_token_idx)

In [21]:
N_EPOCHS = 4
CLIP = 1

best_valid_loss = float('inf')

In [22]:
for epoch in range(N_EPOCHS):
    
    start = time.time()
    # TRAIN 
    #########################################################################
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(train_iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg[:,:-1])
        
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    train_loss = epoch_loss / len(train_iterator)
    #########################################################################
    
    # VALID
    #########################################################################
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(valid_iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg[:,:-1])
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    valid_loss = epoch_loss / len(valid_iterator)
    #########################################################################
    end = time.time()
    
    print(f"EPOCH : {epoch+1}\tTRAIN LOSS : {train_loss:.2f}\tVALID LOSS : {valid_loss:.2f}\tTIME : {end-start:.2f}\n")
    torch.save(model.state_dict(), f'model_{epoch+1}.pt')

EPOCH : 1	TRAIN LOSS : 2.62	VALID LOSS : 1.02	TIME : 763.68

EPOCH : 2	TRAIN LOSS : 0.85	VALID LOSS : 0.79	TIME : 794.35

EPOCH : 3	TRAIN LOSS : 0.59	VALID LOSS : 0.76	TIME : 798.81

EPOCH : 4	TRAIN LOSS : 0.46	VALID LOSS : 0.76	TIME : 796.64



In [23]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break
            
    return trg_indexes[1:]

In [24]:
idxs = random.sample(range(0,len(valid_data.examples)),20)

for i in idxs:
    src = vars(valid_data.examples[i])['src']
    trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence(src, SRC, TRG, model, device)

    print(f"SRC : {' '.join(tokenizer.convert_ids_to_tokens(src))}")
    print(f"TRG : {' '.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PREDICTED : {' '.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context ▁ : ▁abstract . ▁ a ▁much ▁wider ▁variety ▁of ▁biological ▁functions ▁of ▁post re plic ative ▁ d n a ▁ methyl ation ▁is ▁observed ▁in ▁pro k ary o tes ▁than ▁in ▁ e uk ary o tes . ▁in ▁ e uk ary o tes ▁ d n a ▁ methyl ation ▁is ▁ primarily ▁ a ▁means ▁of ▁the ▁control ▁of ▁gene ▁expression . ▁many ▁chapters ▁of ▁this ▁book ▁are ▁ devoted ▁to ▁various ▁aspects ▁of ▁this ▁function . ▁query ▁ : ▁ d n a ▁ methyl ation ▁in ▁ e uk ary o tes
TRG : ▁ a ▁ primarily ▁ a ▁means ▁of ▁the ▁control ▁of ▁gene ▁expression .
PREDICTED : ▁ a ▁much ▁wider ▁variety ▁of ▁biological ▁functions ▁of ▁post re plic ative . </s>

SRC : ▁context ▁ : ▁the ▁is le ▁of ▁sky e ▁is ▁located ▁on ▁the ▁west ▁coast ▁of ▁ s cot l and ▁in ▁the ▁united ▁kingdom ▁of ▁great ▁ b rita in . ▁sky e ▁is ▁one ▁of ▁the ▁most ▁visited ▁parts ▁of ▁the ▁ uk . ▁the ▁island ▁is ▁50 ▁miles ▁long ▁and ▁25 ▁miles ▁wide . ▁the ▁capital ▁is ▁port re e , ▁other ▁villages ▁include ▁du n ve gan , ▁staff in , ▁ u i g , ▁carb ost , ▁